In [1]:
import h5py
import numpy as np
import tensorflow as tf

In [2]:
def load_dataset():
    
    hf=h5py.File("image_final.h5",'r')
    X_train_orig=np.array(hf.get("X_train_orig"))
    X_test_orig=np.array(hf.get("X_test_orig"))
    Y_train_orig=np.array(hf.get("Y_train_orig"))
    Y_test_orig=np.array(hf.get("Y_test_orig"))
    
    Y_train_orig=Y_train_orig.reshape(Y_train_orig.shape[0],1)
    Y_test_orig=Y_test_orig.reshape(Y_test_orig.shape[0],1)
    
    hf.close()
    
    return X_train_orig,X_test_orig,Y_train_orig,Y_test_orig

In [3]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc

from keras.layers import Dropout
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [4]:
def model_nn(input_shape=(64,64,1)):
    
    X_input=Input(input_shape)
    X=Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64,64,1))(X_input)
    X=MaxPooling2D(pool_size=(1,1))(X)
    X=Conv2D(32,(3,3),activation='relu')(X)
    X=MaxPooling2D(pool_size=(1,1))(X)
#32 convolution filters used each of size 3x3
#again
    X=Conv2D(64, (3, 3), activation='relu')(X)
    X=MaxPooling2D(pool_size=(1,1))(X)

#64 convolution filters used each of size 3x3
#choose the best features via pooling
    
#randomly turn neurons on and off to improve convergence
    X=Dropout(0.25)(X)
#flatten since too many dimensions, we only want a classification output
    X=Flatten()(X)
#fully connected to get all relevant data
    X=Dense(128, activation='relu')(X)
#one more dropout for convergence' sake :) 
    X=Dropout(0.5)(X)
#output a softmax to squash the matrix into output probabilities
    X=Dense(1, activation='sigmoid')(X)
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    
    return model

In [5]:
model =model_nn(input_shape = (64, 64, 1))

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
X_train_orig, X_test_orig, Y_train, Y_test = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

In [8]:
model.fit(X_train, Y_train, epochs = 15, batch_size = 32)

Epoch 1/15
83201/83201 [==============================] - 1982s 24ms/step - loss: 0.1945 - accuracy: 0.9244
Epoch 2/15
83201/83201 [==============================] - 2039s 25ms/step - loss: 0.1049 - accuracy: 0.9635
Epoch 3/15
83201/83201 [==============================] - 1905s 23ms/step - loss: 0.0784 - accuracy: 0.9728
Epoch 4/15
83201/83201 [==============================] - 1895s 23ms/step - loss: 0.0690 - accuracy: 0.9761
Epoch 5/15
83201/83201 [==============================] - 1900s 23ms/step - loss: 0.0581 - accuracy: 0.9800
Epoch 6/15
83201/83201 [==============================] - 1948s 23ms/step - loss: 0.0523 - accuracy: 0.9814
Epoch 7/15
83201/83201 [==============================] - 2151s 26ms/step - loss: 0.0477 - accuracy: 0.9828
Epoch 8/15
83201/83201 [==============================] - 1985s 24ms/step - loss: 0.0401 - accuracy: 0.9860
Epoch 9/15
83201/83201 [==============================] - 1993s 24ms/step - loss: 0.0385 - accuracy: 0.9859
Epoch 10/15
83201/83201 [===

In [9]:
y_pred=model.predict(X_test)

In [11]:
y_pred.shape

(1697, 1)

In [11]:
from sklearn.metrics import confusion_matrix as cf

In [12]:
cf(Y_test,y_pred)

array([[827,  11],
       [ 16, 843]], dtype=int64)

In [10]:
y_pred=(y_pred>0.5)

In [15]:
model.save('models/cnnCat2.h5', overwrite=True)